# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

In [216]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from scipy import stats as st 

## Загрузка и подготовка данных

In [217]:
geo_data_0 = pd.read_csv("/datasets/geo_data_0.csv")
geo_data_1 = pd.read_csv("/datasets/geo_data_1.csv")
geo_data_2 = pd.read_csv("/datasets/geo_data_2.csv")

Посмотрим на данные

In [218]:
print(geo_data_0.info())
print(geo_data_1.info())
print(geo_data_2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column  

In [219]:
geo_data_0.drop('id', axis=1, inplace=True)
geo_data_1.drop('id', axis=1, inplace=True)
geo_data_2.drop('id', axis=1, inplace=True)

id т.к.  уникальны нам не нужны

Посмотрим на явные повторы

In [220]:
print(geo_data_0.duplicated().sum())
print(geo_data_1.duplicated().sum())
print(geo_data_2.duplicated().sum())

0
0
0


## Обучение и проверка модели

In [221]:
features_0 = geo_data_0.drop(['product'], axis=1)
target_0 = geo_data_0['product']

features_train_0, features_valid_0, target_train_0, target_valid_0 = train_test_split(
    features_0, target_0, test_size=0.25, random_state=12345)


features_1 = geo_data_1.drop(['product'], axis=1)
target_1 = geo_data_1['product']

features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(
    features_1, target_1, test_size=0.25, random_state=12345)


features_2 = geo_data_2.drop(['product'], axis=1)
target_2 = geo_data_2['product']

features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(
    features_2, target_2, test_size=0.25, random_state=12345)

In [222]:
model = LinearRegression()
model.fit(features_train_0, target_train_0)
predictions_valid_0 = model.predict(features_valid_0)
result_0 = mean_squared_error(target_valid_0, predictions_valid_0) ** 0.5
print("RMSE модели линейной регрессии на валидационной выборке:", result_0)
print('Средний кол-во ресурса:', predictions_valid_0.mean())

RMSE модели линейной регрессии на валидационной выборке: 37.5794217150813
Средний кол-во ресурса: 92.59256778438035


In [223]:
model = LinearRegression()
model.fit(features_train_1, target_train_1)
predictions_valid_1 = model.predict(features_valid_1)
result_1 = mean_squared_error(target_valid_1, predictions_valid_1) ** 0.5
print("RMSE модели линейной регрессии на валидационной выборке:", result_1)
print('Средний кол-во ресурса:', predictions_valid_1.mean())

RMSE модели линейной регрессии на валидационной выборке: 0.893099286775617
Средний кол-во ресурса: 68.728546895446


In [224]:
model = LinearRegression()
model.fit(features_train_2, target_train_2)
predictions_valid_2 = model.predict(features_valid_2)
result_2 = mean_squared_error(target_valid_2, predictions_valid_2) ** 0.5
print("RMSE модели линейной регрессии на валидационной выборке:", result_2)
print('Средний кол-во ресурса:', predictions_valid_2.mean())

RMSE модели линейной регрессии на валидационной выборке: 40.02970873393434
Средний кол-во ресурса: 94.96504596800489


Лучший показатель RMSE у 2 региона.Но запасы малы по сравнению с остальными...

In [225]:
def model_and_split(geo_data):
    features = geo_data.drop(['product'], axis=1)
    target = geo_data['product']

    features_train, features_valid, target_train, target_valid = train_test_split(
        features, target, test_size=0.25, random_state=12345)

    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)

    rmse = mean_squared_error(target_valid, predictions_valid) ** 0.5
    mean = predictions_valid.mean()
    return target_valid, predictions_valid, rmse, mean

In [226]:
target_valid_0, predictions_valid_0, rmse_0, mean_0 = model_and_split(
    geo_data_0)
print("RMSE модели линейной регрессии на валидационной выборке #1:", rmse_0)
print('Средний кол-во ресурса:', mean_0)

RMSE модели линейной регрессии на валидационной выборке #1: 37.5794217150813
Средний кол-во ресурса: 92.59256778438035


In [227]:
target_valid_1, predictions_valid_1, rmse_1, mean_1 = model_and_split(
    geo_data_1)
print("RMSE модели линейной регрессии на валидационной выборке #2:", rmse_1)
print('Средний кол-во ресурса:', mean_1)

RMSE модели линейной регрессии на валидационной выборке #2: 0.893099286775617
Средний кол-во ресурса: 68.728546895446


In [228]:
target_valid_2, predictions_valid_2, rmse_2, mean_2 = model_and_split(
    geo_data_2)
print("RMSE модели линейной регрессии на валидационной выборке #3:", rmse_2)
print('Средний кол-во ресурса:', mean_2)

RMSE модели линейной регрессии на валидационной выборке #3: 40.02970873393434
Средний кол-во ресурса: 94.96504596800489


## Подготовка к расчёту прибыли

При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
Бюджет на разработку скважин в регионе — 10 млрд рублей.
При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

In [229]:
BUDGET = 10000000000
PRODUCT = 450000
WELL = 200

Далее посмотрим,  сколько нам надо ресурса с 1 скважины, что бы выйти в 0

In [230]:
PRICE = BUDGET/WELL  # на 1 скважину
LOW = PRICE/PRODUCT
print(LOW)

111.11111111111111


Получается нам надо с 1 скважины не менее 111,1  барелей.
Зрительно видно, что у нас явно по среднему запасу не один из регионов не проходит.
По заданию надо укзать сколько

In [231]:
print('В 1 регионе не хватает:', abs(predictions_valid_0.mean()-LOW))

В 1 регионе не хватает: 18.518543326730764


In [232]:
print('В 2 регионе не хватает:', abs(predictions_valid_1.mean()-LOW))

В 2 регионе не хватает: 42.38256421566511


In [233]:
print('В 3 регионе не хватает:', abs(predictions_valid_2.mean()-LOW))

В 3 регионе не хватает: 16.146065143106227


Результат не даёт понять о прибыли региона.

## Расчёт прибыли и рисков 

Функция для посчета дохода топ скважин.

In [234]:
def profit(target, probabilities, count):
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target[probs_sorted.index][:count]
    itog = selected.sum()*PRODUCT
    return itog-BUDGET

In [235]:
def profit_2(target, probabilities):
    target = pd.Series(target).reset_index(drop=True)
    probabilities = pd.Series(probabilities)
    state = np.random.RandomState(12345)
    values = []
    for i in range(1000):
        target_subsample = target.sample(500, replace=True, random_state=state)
        probs_subsample = probabilities[target_subsample.index]
        values.append(profit(target_subsample, probs_subsample, 200))
    val = pd.Series(values)
    lower = val.quantile(0.025)
    upper = val.quantile(0.975)
    procent = st.percentileofscore(val, 0)
    mean = val.mean()
    print("Средняя выручка:", mean)
    print("Доверие от:", lower)
    print("Доверие до:", upper)
    print("Риск:", procent, '%')

In [236]:
print('Регион №1')
profit_2(target_valid_0, predictions_valid_0)

Регион №1
Средняя выручка: 425938526.91059244
Доверие от: -102090094.83793654
Доверие до: 947976353.3583689
Риск: 6.0 %


In [237]:
print('Регион №2')
profit_2(target_valid_1, predictions_valid_1)

Регион №2
Средняя выручка: 515222773.4432899
Доверие от: 68873225.37050176
Доверие до: 931547591.2570494
Риск: 1.0 %


In [238]:
print('Регион №3')
profit_2(target_valid_2, predictions_valid_2)

Регион №3
Средняя выручка: 435008362.7827556
Доверие от: -128880547.32978901
Доверие до: 969706954.1802661
Риск: 6.4 %


Написали функции, для выручки, 95%-й доверительный интервал и %риска убытков. 

Выводы:
Изначально запасы 2 региона (68.7)  меньше чем остальных регионов. А также меньше минимального запаса для доходности месторождения . В итоге после наших вычислений , топ регион стал  именно второй – риск убытков всего 1.0 %. 
2 регион рекомендуем для разработки на основании проведенного исследования. Находящийся в датасете ("/datasets/geo_data_1.csv")
